# Normalization
In this file we normalize the data from the previous step (clean_data.csv) and write it to a new csv (norm_data.csv)

## Imports

In [57]:
from IPython import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd

## Reading the CSV
We use pandas to read the csv with the correct options.

In [58]:
df = pd.read_csv('../data/clean_data.csv', header=0, decimal='.')

df.head()

,url,label,url_length,starts_with_ip,url_entropy,has_punycode,digit_letter_ratio,dot_count,at_count,dash_count,tld_count,domain_has_digits,subdomain_count,nan_char_entropy,has_internal_links,domain_age_days
0,prd.dxchcm.com,legitimate,14,False,3.039149,False,0.000000,2,0,0,0,False,1,0.401051,False,1697.0
1,https://appleupdateinformationaccount.serveuse...,phishing,60,False,4.296835,False,0.000000,2,0,0,0,False,1,0.675004,False,8499.0
2,https://alerts-pp.com/PayPal,phishing,28,False,3.913800,False,0.000000,1,0,1,0,False,0,0.688639,False,4281.0
3,klinio.com,legitimate,10,False,2.921928,False,0.000000,1,0,0,0,False,0,0.332193,False,1984.0
4,1d65ec9a.us-west-1.token.awswaf.com,legitimate,35,False,4.071582,False,0.208333,4,0,2,0,False,3,0.357632,False,3261.0


## Copy the df to a work dataframe
we'll be using X as the main data from the df and y for the url and label columns that don't need to be normalised

In [80]:
X = df.copy()

url = X.pop('url')
label = X.pop('label')

y = pd.DataFrame().assign(url=url, label=label)

In [81]:
X
y

,url_length,starts_with_ip,url_entropy,has_punycode,digit_letter_ratio,dot_count,at_count,dash_count,tld_count,domain_has_digits,subdomain_count,nan_char_entropy,has_internal_links,domain_age_days
0,14,False,3.039149,False,0.000000,2,0,0,0,False,1,0.401051,False,1697.0
1,60,False,4.296835,False,0.000000,2,0,0,0,False,1,0.675004,False,8499.0
2,28,False,3.913800,False,0.000000,1,0,1,0,False,0,0.688639,False,4281.0
3,10,False,2.921928,False,0.000000,1,0,0,0,False,0,0.332193,False,1984.0
4,35,False,4.071582,False,0.208333,4,0,2,0,False,3,0.357632,False,3261.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2499987,10,False,2.721928,False,0.000000,1,0,0,0,False,0,0.332193,False,353.0
2499988,45,False,4.144538,False,0.027027,2,0,2,0,False,1,0.521317,False,7890.0
2499989,22,False,3.788755,False,0.000000,2,0,0,0,False,1,0.314494,False,6073.0
2499990,14,False,3.378783,False,0.000000,2,0,0,0,False,1,0.401051,False,2897.0


,url,label
0,prd.dxchcm.com,legitimate
1,https://appleupdateinformationaccount.serveuse...,phishing
2,https://alerts-pp.com/PayPal,phishing
3,klinio.com,legitimate
4,1d65ec9a.us-west-1.token.awswaf.com,legitimate
...,...,...
2499987,lalauni.cn,legitimate
2499988,https://instagram-login-b1aitours.blogspot.be,phishing
2499989,ticker.risedisplay.com,legitimate
2499990,servt.didna.io,legitimate


## Normalising the data
To train the model it is better if most of the values are an int.
Below we check which values are already of type int

In [82]:
discrete_features = X.dtypes == int

discrete_features

url_length             True
starts_with_ip        False
url_entropy           False
has_punycode          False
digit_letter_ratio    False
dot_count              True
at_count               True
dash_count             True
tld_count              True
domain_has_digits     False
subdomain_count        True
nan_char_entropy      False
has_internal_links    False
domain_age_days       False
dtype: bool

We can see that most of the columns are not of type int yet. Below we will convert these

### Convert the boolean types to an int

In [83]:
bools = []

for col in X.select_dtypes('bool'):
    bools.append(col)
    
bools

['starts_with_ip', 'has_punycode', 'domain_has_digits', 'has_internal_links']

The column names listed above have a boolean value. It is better to have these converted to an int (0, 1). We can see in the dataframe below that all the values show False and True

In [84]:
X[['starts_with_ip', 'has_punycode', 'domain_has_digits', 'has_internal_links']]

,starts_with_ip,has_punycode,domain_has_digits,has_internal_links
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
2499987,False,False,False,False
2499988,False,False,False,False
2499989,False,False,False,False
2499990,False,False,False,False


In [85]:
for col in X.select_dtypes("bool"):
    X[col] = X[col].astype(int)

In [86]:
bools = []

for col in X.select_dtypes('bool'):
    bools.append(col)
    
bools

[]

The list now turns up empty so the transformation worked. When we look at the values in X for the columns given in the first list we will now see 0's and 1's

In [87]:
X[['starts_with_ip', 'has_punycode', 'domain_has_digits', 'has_internal_links']]

,starts_with_ip,has_punycode,domain_has_digits,has_internal_links
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
...,...,...,...,...
2499987,0,0,0,0
2499988,0,0,0,0
2499989,0,0,0,0
2499990,0,0,0,0


In [88]:
discrete_features = X.dtypes == int

discrete_features

url_length             True
starts_with_ip         True
url_entropy           False
has_punycode           True
digit_letter_ratio    False
dot_count              True
at_count               True
dash_count             True
tld_count              True
domain_has_digits      True
subdomain_count        True
nan_char_entropy      False
has_internal_links     True
domain_age_days       False
dtype: bool

We can see above that most of the values are now of type int. We can try to convert the string types to an int now

### Convert strings to int
converting string to an int can be done by taking all the unique string values and giving it an int value. Below we will perform this action

In [89]:
objects = []
for col in X.select_dtypes("object"):
    objects.append(col)
    
objects

[]

We can see that the dataset has no object types to be converted, but we will perform the action to be sure. 

In [90]:
for col in X.select_dtypes("object"):
    X[col], _ = X[col].factorize()

## Join the normalised data and the url, label dataframe
After the normalisation we will join the 2 dataframes together again to save it to a new csv

In [93]:
norm_data = pd.concat([y, X], axis=1)

norm_data.head()

,url,label,url_length,starts_with_ip,url_entropy,has_punycode,digit_letter_ratio,dot_count,at_count,dash_count,tld_count,domain_has_digits,subdomain_count,nan_char_entropy,has_internal_links,domain_age_days
0,prd.dxchcm.com,legitimate,14,0,3.039149,0,0.000000,2,0,0,0,0,1,0.401051,0,1697.0
1,https://appleupdateinformationaccount.serveuse...,phishing,60,0,4.296835,0,0.000000,2,0,0,0,0,1,0.675004,0,8499.0
2,https://alerts-pp.com/PayPal,phishing,28,0,3.913800,0,0.000000,1,0,1,0,0,0,0.688639,0,4281.0
3,klinio.com,legitimate,10,0,2.921928,0,0.000000,1,0,0,0,0,0,0.332193,0,1984.0
4,1d65ec9a.us-west-1.token.awswaf.com,legitimate,35,0,4.071582,0,0.208333,4,0,2,0,0,3,0.357632,0,3261.0


## Save the new normalised data to a CSV
We use the option 'index=False' so the index column of the dataset isn't saved to the CSV

In [94]:
norm_data.to_csv('../data/norm_data.csv', index=False)